In [1]:
!pip install -qU \
    openai==1.6.1 \
    pinecone-client==3.1.0 \
    langchain==0.1.1 \
    langchain-community==0.0.13 \
    tiktoken==0.5.2 \
    datasets==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
import os
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass("Enter your OpenAI API key: ")
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

Enter your OpenAI API key: ··········


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
from pinecone import Pinecone

api_key = os.getenv("PINECONE_API_KEY") or getpass("Enter your Pinecone API key: ")

# configure client
pc = Pinecone(api_key=api_key)

Enter your Pinecone API key: ··········


In [5]:
from pinecone import ServerlessSpec
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)

In [7]:
import time

index_name = 'llmtesting'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

if index_name not in existing_indexes:
    pc.create_index(
        index_name,
        dimension=1536,
        metric='dotproduct',
        spec = spec
    )
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('/content/formatted_hse_programs - formatted_hse_programs.csv')

In [10]:
data = df

In [11]:
df

,web-scraper-order,name,formatted_info
0,1713060573-1,География глобальных изменений и геоинформацио...,Бакалаврская программа География глобальных из...
1,1713060573-2,Клеточная и молекулярная биотехнология,Бакалаврская программа Клеточная и молекулярна...
2,1713060573-3,Когнитивная нейробиология,Бакалаврская программа Когнитивная нейробиолог...
3,1713060573-4,Математика,Бакалаврская программа Математика\n\nОписание ...
4,1713060573-5,Совместный бакалавриат НИУ ВШЭ и ЦПМ,Бакалаврская программа Совместный бакалавриат ...
...,...,...,...
58,1713060573-59,Международная программа «Международные отношен...,Бакалаврская программа Международная программа...
59,1713060573-60,Программа двух дипломов НИУ ВШЭ и Университета...,Бакалаврская программа Программа двух дипломов...
60,1713060573-61,Политология,Бакалаврская программа Политология\n\nОписание...
61,1713060573-62,Психология,Бакалаврская программа Психология\n\nОписание ...


In [12]:
from tqdm.auto import tqdm

batch_size = 100

texts = []
metadatas = []

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]
    # first get metadata fields for this record
    metadatas = [{
        'title': record['name'],
        'text': record['formatted_info']
    } for j, record in batch.iterrows()]
    # get the list of contexts / documents
    documents = batch['formatted_info']
    # create document embeddings
    embeds = embed.embed_documents(documents)
    # get IDs
    ids = batch['web-scraper-order']
    # add everything to pinecone
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [14]:
query = "Сколько стоит прикладная математика и информатика?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='Бакалаврская программа Прикладная математика\n\nОписание программы Прикладная математика\nПрограмма направлена на подготовку специалистов, способных решать широкий спектр задач современной науки и техники, опираясь на самые современные достижения в области математики, физики, информационных технологий с использованием средств вычислительной техники.Студенты получают знания в области технологий проектирования динамических систем; технологий обработки, передачи, хранения и защиты информации; технологий разработки и сопровождения специализированного программного обеспечения; технологий математического моделирования сложных физико-механических систем и процессов, включая наноструктуры и космические системы; технологий создания интеллектуальных информационных систем, систем навигации и управления\n\nЯзык обучения на программе Прикладная математика\nОбучение ведется на русском и частично на английском языке\n\nВступительные экзамены на программу Прикладная математика\